<a href="https://colab.research.google.com/github/joepareti54/joepareti54/blob/main/llm_rag_test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install pymupdf
!pip install transformers
!pip install faiss-gpu
!pip install torch  # Ensure PyTorch is installed

import fitz  # PyMuPDF for handling PDFs
import os
import numpy as np
import faiss
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from google.colab import drive
drive.mount('/content/drive')

# Set environment variable to optimize PyTorch CUDA memory allocation
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''.join(page.get_text() for page in doc)
    doc.close()
    return text

# Function to load and process a limited number of documents from a given directory
def load_and_process_documents(directory_path, limit=15):
    documents = []
    for i, filename in enumerate(os.listdir(directory_path)):
        if i >= limit:  # Limit the number of files processed
            break
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                documents.append(text)
    return documents

# Define path to your directory containing PDF files
directory_path = '/content/drive/My Drive/All_Finance_PDF_files_old/'

# Load and process documents (limit to 15)
documents = load_and_process_documents(directory_path)

# Initialize model and tokenizer
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map='auto')
model.eval()  # Set model to evaluation mode

# Function to create embeddings with memory optimization
def create_embeddings(texts, batch_size=2):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        with torch.no_grad():
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(model.device)
            outputs = model(**inputs, output_hidden_states=True)
            batch_embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy()  # Take the first token ([CLS]) embeddings
            embeddings.extend(batch_embeddings)
    return np.array(embeddings)

# Generate embeddings for all loaded documents
embeddings = create_embeddings(documents)

# Setup FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

# Retrieval function based on embeddings
def retrieve_documents(query, k=5):
    query_embedding = create_embeddings([query])[0]  # Get embedding for query
    _, indices = index.search(np.array([query_embedding]), k)
    return [(documents[i], indices[0][i]) for i in indices[0]]

# Example query and retrieval
query = "What is the impact of climate change?"
retrieved_docs = retrieve_documents(query)
for doc, score in retrieved_docs:
    print(doc[:200], "Score:", score)  # Print first 200 characters of each document and its score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.7 MB/s eta 0:00:00
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.54 GiB. GPU 0 has a total capacity of 14.75 GiB of which 289.06 MiB is free. Process 23820 has 14.46 GiB memory in use. Of the allocated memory 14.23 GiB is allocated by PyTorch, and 111.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)